In [1]:
!export LC_ALL=en_US.UTF-8
import tensorflow as tf
import json
from pprint import pprint
import re
import numpy as np
from embedding import *

In [2]:
data = json.load(open('../train-v1.1.json'))

In [3]:
def extractor(data):
    contexts = []
    qas = []
    for i in range(len(data["data"])):
        for j in range(len(data["data"][i]["paragraphs"])):
            contexts.append(data["data"][i]["paragraphs"][j]["context"])
            qas.append(data["data"][i]["paragraphs"][j]["qas"])
    return (contexts,qas)

In [4]:
CapPassage = False

from nltk.tokenize import word_tokenize
contexts,qas = extractor(data)

def find_sub_list(sl,l):
    sll=len(sl)
    for ind in (i for i,e in enumerate(l) if e==sl[0]):
        if l[ind:ind+sll]==sl:
            return ind,ind+sll
    return (-1,-1)

def capPassage(passage,answer,cap_length = 30):
    y = np.zeros(cap_length)
    left,right = find_sub_list(answer,passage)
    if(left==-1):
        return passage[0:cap_length]
    left = left - int((cap_length - len(answer))/2)
    right = right + int((cap_length + len(answer))/2)
    if(left < 0):
        left = 0
    if(right > len(passage)):
        right = len(passage)
    return passage[left:right]
    
def findAnsVec(answer,passage):
    ans = np.zeros((len(passage)))
    start,end = find_sub_list(answer,passage)
    if(start==-1):
        start = passage.index(answer[0])
        end = start + len(answer)
    ans[start:end] = 1
    return ans

In [5]:
X_train_comp_all = []
X_train_comp_ans_all = []
X_train_ans_all = []
Y_train_ques_all = []
invalid = 0
X_train_ans_label_all = []
for i,context in enumerate(contexts):
    passage = word_tokenize(context.lower())
    
    a_lab = np.zeros(len(passage))
    for j,_ in enumerate(qas[i]):
        answer = word_tokenize(qas[i][j]["answers"][0]['text'].lower())
        start,end = find_sub_list(answer,passage)
        if start == -1:
            continue
        a_lab[start:end+1] = 1
            
            
    for j,_ in enumerate(qas[i]):
        try:
            question = word_tokenize(qas[i][j]['question'].lower())
            answer = word_tokenize(qas[i][j]["answers"][0]['text'].lower())
            
            if CapPassage:
                cappedPassage = capPassage(passage,answer)
            else:
                cappedPassage = passage
            
            X_train_comp_ans_all.append(findAnsVec(answer,passage))
            X_train_ans_label_all.append(a_lab)
            X_train_comp_all.append(cappedPassage)
            X_train_ans_all.append(answer)
            Y_train_ques_all.append(question)
        except Exception as e:
            invalid = invalid+1
    

In [6]:
from collections import Counter
import operator
def findKMostFrequentWords(k):
    ctr = Counter([item for sublist in X_train_comp_all for item in sublist] + [item for sublist in Y_train_ques_all for item in sublist])
    sorted_ctr = sorted(ctr.items(), key=operator.itemgetter(1), reverse=True)
    return [item[0] for item in sorted_ctr[0:k]]

In [7]:
wordToTake = 50000
words = findKMostFrequentWords(100000)

In [8]:
_word_to_idx_reduced = {}
_idx_to_word_reduced = []


def _add_word_reduced(word):
    idx = len(_idx_to_word_reduced)
    _word_to_idx_reduced[word] = idx
    _idx_to_word_reduced.append(word)
    return idx


UNKNOWN_TOKEN = _add_word_reduced(UNKNOWN_WORD)
START_TOKEN = _add_word_reduced(START_WORD)
END_TOKEN = _add_word_reduced(END_WORD)




dimensions = glove.shape[1]
reduced_glove = []
reduced_glove.append(np.zeros(dimensions))
reduced_glove.append(-np.ones(dimensions))
reduced_glove.append(np.ones(dimensions))

for word in words:
    l = look_up_word(word)
    if(l != UNKNOWN_TOKEN):
        idx = _add_word_reduced(word)
        reduced_glove.append(glove[l])
        if(len(reduced_glove) == wordToTake):
            break
        
def look_up_word_reduced(word):
    return _word_to_idx_reduced.get(word, UNKNOWN_TOKEN)


def look_up_token_reduced(token):
    return _idx_to_word_reduced[token]

reduced_glove = np.array(reduced_glove)
reduced_glove.shape

(50000, 300)

In [9]:
!hostname
print(invalid)
for i in np.where(X_train_ans_label_all[110] == 1)[0]:
    print(X_train_comp_all[110][i])

cuda2.cims.nyu.edu
1510
rev
.
john
j.
cavanaugh
,
c.s.c
.
served
more
than
half
,
lobund
institute
for
animal
studies
and
medieval
institute
.
hall
of
liberal
arts
(


In [10]:
print(X_train_comp_all[0])
print(X_train_ans_all[0])

['architecturally', ',', 'the', 'school', 'has', 'a', 'catholic', 'character', '.', 'atop', 'the', 'main', 'building', "'s", 'gold', 'dome', 'is', 'a', 'golden', 'statue', 'of', 'the', 'virgin', 'mary', '.', 'immediately', 'in', 'front', 'of', 'the', 'main', 'building', 'and', 'facing', 'it', ',', 'is', 'a', 'copper', 'statue', 'of', 'christ', 'with', 'arms', 'upraised', 'with', 'the', 'legend', '``', 'ad', 'me', 'omnes', "''", '.', 'next', 'to', 'the', 'main', 'building', 'is', 'the', 'basilica', 'of', 'the', 'sacred', 'heart', '.', 'immediately', 'behind', 'the', 'basilica', 'is', 'the', 'grotto', ',', 'a', 'marian', 'place', 'of', 'prayer', 'and', 'reflection', '.', 'it', 'is', 'a', 'replica', 'of', 'the', 'grotto', 'at', 'lourdes', ',', 'france', 'where', 'the', 'virgin', 'mary', 'reputedly', 'appeared', 'to', 'saint', 'bernadette', 'soubirous', 'in', '1858.', 'at', 'the', 'end', 'of', 'the', 'main', 'drive', '(', 'and', 'in', 'a', 'direct', 'line', 'that', 'connects', 'through', '

In [11]:
find_sub_list(X_train_ans_all[0] , X_train_comp_all[0])

(101, 104)

In [12]:
print(invalid)
print(X_train_comp_all[101])
print(X_train_ans_all[101])
print(Y_train_ques_all[101])

c = list(zip(X_train_comp_all,X_train_comp_ans_all, X_train_ans_all, X_train_ans_label_all,Y_train_ques_all))
np.random.shuffle(c)
X_train_comp_all_shuffled,X_train_comp_ans_all_shuffled, X_train_ans_shuffled, X_train_ans_label_shuffled,Y_train_ques_all_shuffled = zip(*c)

print(X_train_comp_all_shuffled[101])
print(X_train_comp_ans_all_shuffled[101])
print(X_train_ans_shuffled[101])
print(X_train_ans_label_shuffled[101])
print(Y_train_ques_all_shuffled[101])


1510
['the', 'success', 'of', 'its', 'football', 'team', 'made', 'notre', 'dame', 'a', 'household', 'name', '.', 'the', 'success', 'of', 'note', 'dame', 'reflected', 'rising', 'status', 'of', 'irish', 'americans', 'and', 'catholics', 'in', 'the', '1920s', '.', 'catholics', 'rallied', 'up', 'around', 'the', 'team', 'and', 'listen', 'to', 'the', 'games', 'on', 'the', 'radio', ',', 'especially', 'when', 'it', 'knocked', 'off', 'the', 'schools', 'that', 'symbolized', 'the', 'protestant', 'establishment', 'in', 'america', '—', 'harvard', ',', 'yale', ',', 'princeton', ',', 'and', 'army', '.', 'yet', 'this', 'role', 'as', 'high-profile', 'flagship', 'institution', 'of', 'catholicism', 'made', 'it', 'an', 'easy', 'target', 'of', 'anti-catholicism', '.', 'the', 'most', 'remarkable', 'episode', 'of', 'violence', 'was', 'the', 'clash', 'between', 'notre', 'dame', 'students', 'and', 'the', 'ku', 'klux', 'klan', 'in', '1924.', 'nativism', 'and', 'anti-catholicism', ',', 'especially', 'when', 'dire

In [50]:
examples_to_take_train = 10000

X_train_comp = X_train_comp_all_shuffled[0:examples_to_take_train]
X_train_comp_ans = X_train_comp_ans_all_shuffled[0:examples_to_take_train]
X_train_ans = X_train_ans_shuffled[0:examples_to_take_train]
X_train_ans_label = X_train_ans_label_shuffled[0:examples_to_take_train]
Y_train_ques = Y_train_ques_all_shuffled[0:examples_to_take_train]
answer_indices = [np.where(x==1)[0].tolist() for x in X_train_comp_ans]

In [51]:
max_document_len = len(max(X_train_comp,key=len))
max_answer_len = len(max(X_train_ans,key=len))
max_question_len = len(max(Y_train_ques,key=len)) + 1

In [52]:
document_tokens = np.zeros((examples_to_take_train, max_document_len), dtype=np.int32)
document_lengths = np.zeros(examples_to_take_train, dtype=np.int32)
answer_labels = np.zeros((examples_to_take_train, max_document_len), dtype=np.int32)
answer_masks = np.zeros((examples_to_take_train, max_answer_len, max_document_len), dtype=np.int32)
answer_lengths = np.zeros(examples_to_take_train, dtype=np.int32)
question_input_tokens = np.zeros((examples_to_take_train, max_question_len), dtype=np.int32)
question_output_tokens = np.zeros((examples_to_take_train, max_question_len), dtype=np.int32)
question_lengths = np.zeros(examples_to_take_train, dtype=np.int32)
suppression_answer = np.zeros((examples_to_take_train, reduced_glove.shape[0], 1),dtype=np.int32)
print(suppression_answer.shape)

(10000, 50000, 1)


In [53]:
print(answer_labels.shape)
for i in range(examples_to_take_train):
    answer_labels[i,0:len(X_train_ans_label[i])] = X_train_ans_label[i]
    for j, word in enumerate(X_train_comp[i]):
        document_tokens[i, j] = look_up_word_reduced(word)
    document_lengths[i] = len(X_train_comp[i])

    for j, index in enumerate(answer_indices[i]):
        answer_masks[i, j, index] = 1
    answer_lengths[i] = len(answer_indices[i])
    
    #print(Y_train_ques[i])
    question_input_words = ([START_WORD] + Y_train_ques[i])
    question_output_words = (Y_train_ques[i] + [END_WORD])

    for j, word in enumerate(question_input_words):
            question_input_tokens[i, j] = look_up_word_reduced(word)
    for j, word in enumerate(question_output_words):
        question_output_tokens[i, j] = look_up_word_reduced(word)
    question_lengths[i] = len(question_input_words)
    
    for j, word in enumerate(X_train_ans[i]):
        if(word not in Y_train_ques[i]):
            suppression_answer[i, look_up_word_reduced(word),:] = 1

(10000, 766)


In [54]:
def create_vocabulary(data):
    flat_list = [item for sublist in data for item in sublist]
    vocabulary = sorted(set(flat_list))
    vocabulary.append("<UNK>")
    vocabulary.append("unk")
    vocabulary.append("eos")
    vocabulary = ["<EOS>"] + vocabulary
    word_to_index = { word:i for i,word in enumerate(vocabulary) }
    index_to_word = { i:word for i,word in enumerate(vocabulary) }
    return (vocabulary,word_to_index,index_to_word)

In [55]:
print(reduced_glove.shape)
vocabulary_comp,word_to_index_comp,index_to_word_comp = create_vocabulary(X_train_comp + Y_train_ques)
print(len(vocabulary_comp))
print(word_to_index_comp["?"])
print(word_to_index_comp["what"])

(50000, 300)
62106
5834
59802


In [56]:
def create_one_hot_vector(data,vocabulary,word_to_index,index_to_word, maxLen):
    one_hot = np.zeros([maxLen,len(vocabulary)])
    for i,word in enumerate(data):
        if i >= maxLen:
            break
        if(word not in word_to_index):
            word = "<UNK>"
        one_hot[i][word_to_index[word]] = 1
    return one_hot

def create_one_hot_vector_from_indices(data,maxLen,vocabulary):
    one_hot = np.zeros([maxLen,len(vocabulary)])
    for i,indice in enumerate(data):
        if i >= maxLen:
            break
        one_hot[i][int(indice)] = 1
    return one_hot


def create_one_hot_training_Set(data,maxLen,vocabulary):
    one_hot_data = np.zeros([data.shape[0],maxLen,len(vocabulary)])
    for i in range(data.shape[0]):
        one_hot_data[i] = create_one_hot_vector_from_indices(data[i],maxLen,vocabulary)
    return one_hot_data




In [57]:
def sentences_to_indices_glove(X,max_len):
    
    m = len(X)                                 
    
    X_indices = np.full([m,max_len],look_up_word_reduced(END_WORD))
    
    for i in range(m):
        j = 0
        for w in X[i]:
            if(j>=max_len):
                break;
            
            X_indices[i, j] = look_up_word_reduced(w)
            j = j+1
    return X_indices

In [58]:
document_tokens = sentences_to_indices_glove(X_train_comp, max_document_len)

In [59]:
document_tokens[0]

array([   22,  1795,   385, 19053,     4,  2629,  4553,  7562, 30963,
          35,  3788, 16025,   139,     3,  2592,     5, 24499,    31,
           3,  4237,     5,     3,    65,  4329,   104,     4,     3,
        2493,   167,     8, 19252,     6,     3, 16025,  6960,     7,
        5149,     3,   167,     7,     3,  5646,     4,    51,  2760,
         482,     3,  2592,     5, 24499,     7,  7018,  1734,    91,
       28738,     4,   601,  4932,     4,     7,     3,   700,     5,
           3,   911,     6,    48,    85,   110,  2355,   464,    22,
           3,   545,  9146,     9,  2582,   765,     4,     3, 23165,
           5,     7,  9146,     9,     3,  4329,     4,     3, 20042,
           4,  5304,  8759, 14065,     4, 15324,     4,   338,    22,
           3,   700,     5,     3,   911,     4,     3,  5357,   157,
           4, 22835,     7, 13628,     4,     3,   393,     5,  2276,
        2427,     8,   277,  2642,     4,     3,   855,    73,  1376,
           7,     3,

In [61]:
answer_labels.shape

(10000, 766)

## Model

In [100]:
cellFlag = 'LSTM'

### Document Embedding

In [173]:
import tensorflow.contrib.seq2seq as seq2seq

tf.reset_default_graph()

embedding = tf.get_variable("embedding", initializer=reduced_glove)
embedding = tf.cast(embedding, dtype=tf.float32)
EMBEDDING_DIMENS = glove.shape[1]

d_tokens = tf.placeholder(tf.int32, shape=[None, None], name="d_tokens")
d_lengths = tf.placeholder(tf.int32, shape=[None], name="d_lengths")


document_emb = tf.nn.embedding_lookup(embedding, d_tokens, name="document_emb")
document_emb = tf.cast(document_emb, dtype=tf.float64, name="casted_document_emb")


### Answer Generation

In [174]:
if cellFlag == 'LSTM':
    forward_cell = tf.contrib.rnn.LSTMCell(EMBEDDING_DIMENS)
    backward_cell = tf.contrib.rnn.LSTMCell(EMBEDDING_DIMENS)
elif cellFlag == 'GRU':
    forward_cell = tf.contrib.rnn.GRUCell(EMBEDDING_DIMENS)
    backward_cell = tf.contrib.rnn.GRUCell(EMBEDDING_DIMENS)

answer_outputs, states = tf.nn.bidirectional_dynamic_rnn(
    forward_cell, backward_cell, document_emb, d_lengths, dtype=tf.float64,
    scope="answer_rnn")

answer_outputs = tf.concat(answer_outputs, 2, name="answer_output_concat")

answer_outputs = tf.cast(answer_outputs,tf.float32, name="answer_output_concat")

answer_tags = tf.layers.dense(inputs=answer_outputs, units=2, name="answer_tags")


a_labels = tf.placeholder(tf.int32, shape=[None, None], name="a_labels")

answer_mask = tf.sequence_mask(d_lengths, dtype=tf.float32, name="answer_mask")

answer_loss = seq2seq.sequence_loss(
    logits=answer_tags, targets=a_labels, weights=answer_mask, name="answer_loss")

## Question Generation

### Encoder

In [175]:
encoder_input_mask = tf.placeholder(
    tf.float32, shape=[None, None, None], name="encoder_input_mask")
encoder_inputs = tf.matmul(encoder_input_mask, answer_outputs, name="encoder_inputs")
encoder_lengths = tf.placeholder(tf.int32, shape=[None], name="encoder_lengths")

if cellFlag == 'GRU':
    encoder_cell = tf.contrib.rnn.GRUCell(forward_cell.state_size + backward_cell.state_size)
elif cellFlag == 'LSTM':
    encoder_cell = tf.contrib.rnn.LSTMCell(2 * EMBEDDING_DIMENS)



In [180]:
print(encoder_state.h)

Tensor("encoder_rnn/while/Exit_3:0", shape=(?, 600), dtype=float32)


In [177]:
_, encoder_state = tf.nn.dynamic_rnn(
    encoder_cell, encoder_inputs, encoder_lengths, dtype=tf.float32, scope="encoder_rnn")

In [176]:
encoder_cell.state_size

LSTMStateTuple(c=600, h=600)

### Decoder

In [181]:
from tensorflow.python.layers.core import Dense


decoder_inputs = tf.placeholder(tf.int32, shape=[None, None], name="decoder_inputs")
decoder_lengths = tf.placeholder(tf.int32, shape=[None], name="decoder_lengths")

decoder_emb = tf.nn.embedding_lookup(embedding, decoder_inputs,name="decoder_embedding")
decoder_emb = tf.cast(decoder_emb,tf.float32,name="decoder_embedding_cast")

helper = seq2seq.TrainingHelper(decoder_emb , decoder_lengths, name="helper")


projection = Dense(embedding.shape[0], use_bias=False, name="projection")

if cellFlag == 'GRU':
    decoder_cell = tf.contrib.rnn.GRUCell(encoder_cell.state_size)
elif cellFlag == "LSTM":
    decoder_cell = tf.contrib.rnn.LSTMCell(2 * EMBEDDING_DIMENS)

In [182]:
decoder = seq2seq.BasicDecoder(decoder_cell, helper, encoder_state, output_layer=projection)
decoder_outputs, _, _ = seq2seq.dynamic_decode(decoder, scope="decoder")
decoder_outputs = decoder_outputs.rnn_output

decoder_labels = tf.placeholder(tf.int64, shape=[None, None], name="decoder_labels")
question_mask = tf.sequence_mask(decoder_lengths ,dtype=tf.float32)

## Question Generation Loss

In [183]:
# NLL Loss
question_loss = seq2seq.sequence_loss(
    logits=decoder_outputs, targets=decoder_labels, weights=question_mask,
    name="question_loss")


#Suppression Loss
s_answer = tf.placeholder(tf.float32, shape=[None,None,None], name="suppression_answer")
lambdaSuppress = 0.5
decoder_outputs = tf.Print(decoder_outputs, [decoder_outputs], message="This is decoder_outputs: ")
suppression_loss = lambdaSuppress * tf.reduce_sum(tf.matmul(tf.exp(decoder_outputs), s_answer))
suppression_loss = tf.Print(suppression_loss, [suppression_loss], message="This is suppression_loss: ")

In [184]:
loss = tf.add(answer_loss, question_loss, name="loss")
loss = tf.add(loss, suppression_loss, name="loss")

In [30]:
#document_tokens = np.zeros((examples_to_take_train, max_document_len), dtype=np.int32)
#document_lengths = np.zeros(examples_to_take_train, dtype=np.int32)
#answer_labels = np.zeros((examples_to_take_train, max_document_len), dtype=np.int32)
#answer_masks = np.zeros((examples_to_take_train, max_answer_len, max_document_len), dtype=np.int32)
#answer_lengths = np.zeros(examples_to_take_train, dtype=np.int32)
#question_input_tokens = np.zeros((examples_to_take_train, max_question_len), dtype=np.int32)
#question_output_tokens = np.zeros((examples_to_take_train, max_question_len), dtype=np.int32)
#question_lengths = np.zeros(examples_to_take_train, dtype=np.int32)
#suppression_answer = np.zeros((examples_to_take_train,max_answer_len),dtype=np.int32)

In [31]:
def shuffle_list(*ls):
    l =list(zip(*ls))
    np.random.shuffle(l)
    return zip(*l)

In [185]:
import math
def createBatch(inputs,batch_size,shuffle=False):
    outputs = []
    num_batches = math.ceil(len(inputs[0])/batch_size)
    
    for index,inp in enumerate(inputs):
        start = 0
        output = []
        for i in range(num_batches-1):
            maxD = max(inputs[1][start:start+batch_size])
            maxA = max(inputs[4][start:start+batch_size])
            maxQ = max(inputs[7][start:start+batch_size])
            if index == 0 or index == 2:
                output.append(inp[start:start+batch_size,0:maxD]) 
            elif index==3:
                output.append(inp[start:start+batch_size,0:maxA,0:maxD]) 
            elif index==5 or index==6:
                output.append(inp[start:start+batch_size,0:maxQ])
            elif index == 8: # 
                output.append(inp[start:start+batch_size,:,:]) 
            else: 
                output.append(inp[start:start+batch_size])
            start = start + batch_size
        
        # Remaining training sample i.e. training mod batch_size
        maxD = max(inputs[1][start:])
        maxA = max(inputs[4][start:])
        maxQ = max(inputs[7][start:])
        if index == 0 or index == 2:
            output.append(inp[start:,0:maxD]) 
        elif index==3:
            output.append(inp[start:,0:maxA,0:maxD]) 
        elif index==5 or index==6:
            output.append(inp[start:,0:maxQ]) 
        elif index ==8:
            output.append(inp[start:,:, :]) 
        else: 
            output.append(inp[start:])
        outputs.append(output)
    
    return outputs


In [188]:
batch_size = 32
batch_input = createBatch([document_tokens,document_lengths,answer_labels,answer_masks,answer_lengths,question_input_tokens,question_output_tokens,question_lengths,suppression_answer]
                    ,batch_size)

In [189]:
"No of batches:",len( batch_input[0])

('No of batches:', 313)

In [ ]:
optimizer = tf.train.AdamOptimizer(learning_rate=3e-3).minimize(loss)

saver = tf.train.Saver()
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.InteractiveSession(config=config)

session.run(tf.global_variables_initializer())
# session.run(tf.variables_initializer(saved_vars))


EPOCHS = 20

for epoch in range(1, EPOCHS + 1):
    print("Epoch {0}".format(epoch))
    for batchNum in range(len(batch_input[0])):
        print("Batch : ",batchNum)
        t = session.run([optimizer, loss, decoder_outputs, suppression_loss], {
            d_tokens: batch_input[0][batchNum],
            d_lengths: batch_input[1][batchNum],
            a_labels: batch_input[2][batchNum],
            encoder_input_mask: batch_input[3][batchNum],
            encoder_lengths: batch_input[4][batchNum],
            decoder_inputs: batch_input[5][batchNum],
            decoder_labels: batch_input[6][batchNum],
            decoder_lengths: batch_input[7][batchNum],
            s_answer: batch_input[8][batchNum],
        })
        print("Loss: {0}".format(t[1]))
    if(epoch%5 == 0):
        print("Saving model")
        #saver.save(session, "qgen-model")


Epoch 1
Batch :  0
Loss: 1069.5816650390625
Batch :  1
Loss: 977.320556640625
Batch :  2
Loss: 419.386962890625
Batch :  3
Loss: 819.1289672851562
Batch :  4
Loss: 406.09136962890625
Batch :  5
Loss: 393.1398620605469
Batch :  6
Loss: 245.859130859375
Batch :  7
Loss: 285.4755859375
Batch :  8
Loss: 224.34027099609375
Batch :  9
Loss: 58.62446212768555
Batch :  10
Loss: 254.43963623046875
Batch :  11
Loss: 141.78228759765625
Batch :  12
Loss: 73.60932922363281
Batch :  13
Loss: 301.4520263671875
Batch :  14
Loss: 116.12511444091797
Batch :  15
Loss: 55.39506912231445
Batch :  16
Loss: 222.0098876953125
Batch :  17
Loss: 82.1900634765625
Batch :  18
Loss: 78.52726745605469
Batch :  19
Loss: 56.948158264160156
Batch :  20
Loss: 62.54273223876953
Batch :  21
Loss: 45.830867767333984
Batch :  22
Loss: 85.80056762695312
Batch :  23
Loss: 19.57417869567871
Batch :  24
Loss: 33.038509368896484
Batch :  25
Loss: 43.18663787841797
Batch :  26
Loss: 61.19582748413086
Batch :  27
Loss: 30.8002185

Loss: 7.141891956329346
Batch :  225
Loss: 7.27922248840332
Batch :  226
Loss: 7.382074356079102
Batch :  227
Loss: 7.54067325592041
Batch :  228
Loss: 7.6405720710754395
Batch :  229
Loss: 7.702033042907715
Batch :  230
Loss: 7.076685905456543
Batch :  231
Loss: 7.32241678237915
Batch :  232
Loss: 7.936300277709961
Batch :  233
Loss: 7.358598709106445
Batch :  234
Loss: 7.489475250244141
Batch :  235
Loss: 7.19539213180542
Batch :  236
Loss: 7.758790016174316
Batch :  237
Loss: 8.009986877441406
Batch :  238
Loss: 7.802896499633789
Batch :  239
Loss: 6.998081684112549
Batch :  240
Loss: 7.184821128845215
Batch :  241
Loss: 7.107460975646973
Batch :  242
Loss: 7.66880464553833
Batch :  243
Loss: 7.034924030303955
Batch :  244
Loss: 7.411591053009033
Batch :  245
Loss: 7.551497459411621
Batch :  246
Loss: 7.055408000946045
Batch :  247
Loss: 7.2994561195373535
Batch :  248
Loss: 7.537991523742676
Batch :  249
Loss: 7.435757637023926
Batch :  250
Loss: 7.13217306137085
Batch :  251
Loss:

Loss: 6.255951881408691
Batch :  137
Loss: 6.501692771911621
Batch :  138
Loss: 6.581567764282227
Batch :  139
Loss: 6.7375264167785645
Batch :  140
Loss: 6.394173622131348
Batch :  141
Loss: 6.724205493927002
Batch :  142
Loss: 6.7190093994140625
Batch :  143
Loss: 6.411349296569824
Batch :  144
Loss: 6.999265670776367
Batch :  145
Loss: 6.5244927406311035
Batch :  146
Loss: 6.767501354217529
Batch :  147
Loss: 6.7523980140686035
Batch :  148
Loss: 6.9549479484558105
Batch :  149
Loss: 6.6109795570373535
Batch :  150
Loss: 6.711129188537598
Batch :  151
Loss: 6.5176591873168945
Batch :  152
Loss: 6.588394641876221
Batch :  153
Loss: 6.421726703643799
Batch :  154
Loss: 6.941173076629639
Batch :  155
Loss: 6.745369911193848
Batch :  156
Loss: 6.522328853607178
Batch :  157
Loss: 6.51903772354126
Batch :  158
Loss: 6.646259784698486
Batch :  159
Loss: 6.2275285720825195
Batch :  160
Loss: 6.637674331665039
Batch :  161
Loss: 6.670886039733887
Batch :  162
Loss: 6.758313179016113
Batch :

Loss: 6.057738780975342
Batch :  47
Loss: 6.218738555908203
Batch :  48
Loss: 6.443546295166016
Batch :  49
Loss: 6.1426777839660645
Batch :  50
Loss: 6.352998733520508
Batch :  51
Loss: 6.621384143829346
Batch :  52
Loss: 6.137950420379639
Batch :  53
Loss: 5.965452671051025
Batch :  54
Loss: 6.2577104568481445
Batch :  55
Loss: 5.940954208374023
Batch :  56
Loss: 6.438726425170898
Batch :  57
Loss: 6.123147487640381
Batch :  58
Loss: 6.248194217681885
Batch :  59
Loss: 6.376138687133789
Batch :  60
Loss: 6.162285804748535
Batch :  61
Loss: 6.2060866355896
Batch :  62
Loss: 6.156921863555908
Batch :  63
Loss: 6.117175102233887
Batch :  64
Loss: 6.279160976409912
Batch :  65
Loss: 6.3153181076049805
Batch :  66
Loss: 6.2412590980529785
Batch :  67
Loss: 5.984312534332275
Batch :  68
Loss: 6.487259387969971
Batch :  69
Loss: 6.015294551849365
Batch :  70
Loss: 6.20359468460083
Batch :  71
Loss: 6.406589031219482
Batch :  72
Loss: 6.2306694984436035
Batch :  73
Loss: 6.051054000854492
Ba

Loss: 6.119964599609375
Batch :  270
Loss: 5.754729747772217
Batch :  271
Loss: 5.935001373291016
Batch :  272
Loss: 6.035041332244873
Batch :  273
Loss: 5.941882133483887
Batch :  274
Loss: 6.335686206817627
Batch :  275
Loss: 6.381246566772461
Batch :  276
Loss: 5.634225845336914
Batch :  277
Loss: 6.082145690917969
Batch :  278
Loss: 6.041434288024902
Batch :  279
Loss: 5.968863010406494
Batch :  280
Loss: 6.304779052734375
Batch :  281
Loss: 5.952790260314941
Batch :  282
Loss: 5.857866287231445
Batch :  283
Loss: 6.179609298706055
Batch :  284
Loss: 6.11755895614624
Batch :  285
Loss: 6.248010635375977
Batch :  286
Loss: 6.363430976867676
Batch :  287
Loss: 6.169281482696533
Batch :  288
Loss: 6.135061264038086
Batch :  289
Loss: 5.878298282623291
Batch :  290
Loss: 6.069104194641113
Batch :  291
Loss: 6.06389045715332
Batch :  292
Loss: 6.023507118225098
Batch :  293
Loss: 5.903846263885498
Batch :  294
Loss: 5.547488689422607
Batch :  295
Loss: 6.106835842132568
Batch :  296
Los

In [74]:
saver = tf.train.Saver()
saver.save(session, "qgen-model")

'qgen-model'

In [ ]:
batch_input[3][0].shape

In [88]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)
saver = tf.train.Saver()

saver.restore(session, 'qgen-model')

INFO:tensorflow:Restoring parameters from qgen-model


In [89]:
answers = session.run(answer_tags, {
    d_tokens: batch_input[0][48],
    d_lengths: batch_input[1][48],
})
answers = np.argmax(answers, 2)

In [ ]:
for i in range(28):
    print("Prediction")
    printAllAns(answers,48,0)
    print("Ground Truth")
    printAllAns(batch_input[2][48],48,0)

In [90]:
def printDoc(batch,num):
    for i in batch_input[0][batch][num]:
        print(look_up_token_reduced(i),sep=" ", end=" ")
    print(" ")

def printQues(batch,num):
    for i in batch_input[5][batch][num]:
        print(look_up_token_reduced(i),sep=" ", end=" ")
    print(" ")
    
def printAnsForQuestion(batch, num):
    for i in batch_input[5][batch][num]:
        print(look_up_token_reduced(i),sep=" ", end=" ")
    print(" ")
    
def printAllAns(answers, batch, num):
    for i,word in enumerate(batch_input[0][batch][num]):
        if answers[num][i] == 1 :
            print(look_up_token_reduced(word),sep=" ", end=" ")
    print(" ")
    

In [95]:
import itertools

batchNum = 234

helper = seq2seq.GreedyEmbeddingHelper(embedding, tf.fill([batch_input[0][batchNum].shape[0]], START_TOKEN), END_TOKEN)
decoder = seq2seq.BasicDecoder(decoder_cell, helper, encoder_state, output_layer=projection)
decoder_outputs, _, _ = seq2seq.dynamic_decode(decoder, maximum_iterations=max_question_len)
decoder_outputs = decoder_outputs.rnn_output


questions = session.run(decoder_outputs, {
    d_tokens: batch_input[0][batchNum],
    d_lengths: batch_input[1][batchNum],
    a_labels: batch_input[2][batchNum],
    encoder_input_mask: batch_input[3][batchNum],
    encoder_lengths: batch_input[4][batchNum],
})



In [96]:
batch_input[0][batchNum].shape

(32, 235)

In [97]:
#questions[:,:,END_TOKEN] = 0
qs = np.argmax(questions, 2)

In [98]:
# q = set()
l = []
for i in range(batch_input[0][batchNum].shape[0]):
    print("---------------------------------------------------------------------------------------------")
    question = itertools.takewhile(lambda t: t != END_TOKEN, qs[i])
    print("Generated Question: " + " ".join(look_up_token_reduced(token) for token in question))
    print("Ground Truth Question: ")
    printQues(batchNum,i)
    print("Ground Truth Answer: ", X_train_ans_shuffled[batch_size*batchNum + i])
    print("Context:")
    printDoc(batchNum,i)
    print("---------------------------------------------------------------------------------------------")
    
    
# q

---------------------------------------------------------------------------------------------
Generated Question: who was the name of the treaty developed that the island of the treaty ?
Ground Truth Question: 
<START> which company filed suit against harding , <UNK> , <UNK> & <UNK> 's client ? <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK>  
Ground Truth Answer:  ['healthcare', 'advocates']
Context:
in 2003 , harding <UNK> <UNK> & <UNK> defended a client from a trademark dispute using the archive 's wayback machine . the attorneys were able to demonstrate that the claims made by the plaintiff were invalid , based on the content of their web site from several years prior . the plaintiff , healthcare advocates , then amended their complaint to include the internet archive , accusing the organization of copyright infringement as well as violations of the dmca and the computer fraud and abuse act . healthcare advocates claimed that , since they had installed a robots.txt file on th

seattle is home to the university of washington , as well as the institution 's professional and continuing education unit , the university of washington educational outreach . a study by newsweek international in 2006 cited the university of washington as the twenty-second best university in the world . seattle also has a number of smaller private universities including seattle university and seattle pacific university , the former a jesuit catholic institution , the latter free methodist ; universities aimed at the working adult , like city university and antioch university ; colleges within the seattle colleges district system , comprising north , central , and south ; seminaries , including western seminary and a number of arts colleges , such as cornish college of the arts , pratt fine arts center , and the art institute of seattle . in 2001 , time magazine selected seattle central community college as community college of the year , stating the school `` pushes diverse students t

<START> what did the english east india company have difficulty attracting ? <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK>  
Ground Truth Answer:  ['new', 'immigrants']
Context:
between january and may 1673 , the dutch east india company forcibly took the island , before english reinforcements restored english east india company control . the company experienced difficulty attracting new immigrants , and sentiments of unrest and rebellion fomented among the inhabitants . ecological problems , including deforestation , soil erosion , vermin and drought , led governor isaac <UNK> to suggest in 1715 that the population be moved to mauritius , but this was not acted upon and the company continued to subsidise the community because of the island 's strategic location . a census in 1723 recorded 1,110 people , including 610 slaves . <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <EN

cultural barriers can also keep a person from telling someone they are in pain . religious beliefs may prevent the individual from seeking help . they may feel certain pain treatment is against their religion . they may not report pain because they feel it is a sign that death is near . many people fear the stigma of addiction and avoid pain treatment so as not to be prescribed potentially <UNK> drugs . many asians do not want to lose respect in society by admitting they are in pain and need help , believing the pain should be borne in silence , while other cultures feel they should report pain right away and get immediate relief . gender can also be a factor in reporting pain . sexual differences can be the result of social and cultural expectations , with women expected to be emotional and show pain and men stoic , keeping pain to themselves . <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END

during 1976 , queen played one of their most famous gigs , a free concert in hyde park , london . a concert organised by the entrepreneur richard <UNK> , it set an attendance record with 150,000 people confirmed in the audience . on 1 december 1976 , queen were the intended guests on london 's early evening today programme , but they pulled out at the last-minute , which saw their late replacement on the show , emi <UNK> the sex pistols , give their seminal interview . during the a day at the races tour in 1977 , queen performed sold-out shows at madison square garden , new york , in february , and earls court , london , in june . <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <

on 25 may 2011 the swiss government announced that it plans to end its use of nuclear energy in the next 2 or 3 decades . `` the government has voted for a <UNK> because we want to ensure a secure and autonomous supply of energy '' , energy minister doris <UNK> said that day at a press conference in bern . `` fukushima showed that the risk of nuclear power is too high , which in turn has also increased the costs of this energy form . '' the first reactor would reportedly be taken offline in 2019 and the last one in <UNK> parliament will discuss the plan in june 2011 , and there could be a referendum as well . <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END>

In [ ]:
batch_input[5][18].shape

In [ ]:
batch_input[5][1].shape

In [ ]:
batch_input[6][1].shape

In [ ]:
max(batch_input[7][1])

In [ ]:
max_question_len

In [ ]:
len(batch_input)

In [ ]:
reduced_glove